### Алгоритм MinHash

In [18]:
# Создание класса по разделению текста на shingles (части)
class shingler:
    def __init__(self, k):
        if k > 0:
            self.k = int(k)
        else:
            self.k = 10   
    #inner class utility
    def process_doc(self, document):
        return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()
        document= self.process_doc(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

In [22]:
import hashlib
import re
import math
# Применение hash-функции sha1 для формирования хэшей для частей текста
class hashFamily:
    def __init__(self, i):
        self.resultSize = 8 # размер хэша
        self.maxLen = 20 # длина хэша - сколько разных хэшей вычисляется
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)

In [16]:
from random import randint, seed
class minhashSigner:
    def __init__(self, sig_size):
        self.sig_size=sig_size
        # Инициализация рандомного набора хэш-функции
        self.hash_functions = [hashFamily(randint(0,10000000000)) for i in range(0,sig_size)]

    def compute_set_signature(self, set_):
        # Для каждого токена вычисляются хэши с использованием хэш-функций.
        # Для каждой хэш-функции сохраняется минимальное значение хэша для данного токена.
        set_sig = []
        for h_funct in self.hash_functions:
            min_hash = math.inf
            for el in set_:
                h = h_funct.get_hash_value(el)
                if h < min_hash:
                    min_hash = h

            set_sig.append(min_hash)

        return set_sig
    # возвращает список списков, который можно рассматривать как матрицу сигнатур (подписей)
    def compute_signature_matrix(self, set_list):
        signatures = []
        for s in set_list:
            signatures.append( self.compute_set_signature(s) )
        return signatures

In [34]:
doc = "This is a sample text for testing minhash algorithm"
doc2 = "Sample text for testing the minhash algorithm"

# Разбиваем текст на сэмплы для каждого текста (параметр 10 - обозначает кол-во символов в сэмпле)
shingler_inst = shingler(10)
shinglings = shingler_inst.get_shingles(doc)
shinglings2 = shingler_inst.get_shingles(doc2)
print(shinglings)
print('----------------------------------------------------')

# Вычисляем хэш-функцию для каждого сэмпла и перем минимальное значение (размер должен быть равен 20 - кол-во взятых minhash)
minhash_sig = minhashSigner(20).compute_signature_matrix([shinglings,shinglings2])
print(minhash_sig)
print('----------------------------------------------------')

set_sig_1 = set(minhash_sig[0])
set_sig_2 = set(minhash_sig[1])
print(set_sig_1)
print(set_sig_2)
print('----------------------------------------------------')

# Вычисляем схожесть через индекс Жакара
jaccard_similarity_sig = len(set_sig_1.intersection(set_sig_2))/len(set_sig_1.union(set_sig_2))
print(jaccard_similarity_sig)
print('----------------------------------------------------')

# Вычисляем схожесть через индекс Жакара у shingles
jaccard_similarity_shingle_set = len(set(shinglings).intersection(shinglings2))/len(set(shinglings).union(shinglings2))
print(jaccard_similarity_shingle_set)
print('----------------------------------------------------')

{'sample tex', ' text for ', 'a sample t', 'or testing', 'sting minh', 'ple text f', 's a sample', 'is is a sa', ' a sample ', ' for testi', 'ext for te', 'his is a s', 'testing mi', 'is a sampl', ' is a samp', 't for test', ' minhash a', 'ing minhas', 'ting minha', 'this is a ', 'xt for tes', 'h algorith', ' sample te', 'ample text', 'ng minhash', 'mple text ', 'for testin', 'minhash al', ' testing m', 'esting min', 'g minhash ', 'inhash alg', 'hash algor', ' algorithm', 'text for t', 'e text for', 'le text fo', 'sh algorit', 'r testing ', 'ash algori', 'nhash algo', 's is a sam'}
----------------------------------------------------
[[48294715, 135952987, 67559240, 203835105, 223411848, 167838644, 151908201, 95721586, 53735794, 157135870, 98146585, 43453560, 14381977, 269643867, 107362640, 29245735, 15001205, 49334622, 17333750, 37248475], [48294715, 51008138, 72325829, 203835105, 532266030, 167838644, 54658253, 95721586, 112623961, 171130964, 98146585, 163432405, 14381977, 65086000, 

Можно заметить, что сходство Jaccard между сигнатурами (сэмплами) документов близко к сходству между наборами shingle